In [1]:
import sys
import os
import pandas as pd
import ECAUGT as HCAd_Client
import time
import multiprocessing
import numpy as np

In [2]:
# set parameters
endpoint = "https://HCAd-Datasets2.cn-beijing.ots.aliyuncs.com"
access_id = "LTAI5tHMCadpjLSJsik8AjXg" #enter your id and keys
access_key = "gRFJlZnKWpqOAOQWqWYcK7CMG34xFK"
instance_name = "HCAd-Datasets"
table_name = 'HCA_d'

In [3]:
# # setup client
HCAd_Client.Setup_Client(endpoint, access_id, access_key, instance_name, table_name)

Connected to the server, find the table.
HCA_d
TableName: HCA_d
PrimaryKey: [('cid', 'INTEGER')]
Reserved read throughput: 0
Reserved write throughput: 0
Last increase throughput time: 1605795297
Last decrease throughput time: None
table options's time to live: -1
table options's max version: 1
table options's max_time_deviation: 86400


0

In [4]:
hcad_organs = ['Placenta_ChorionicVillus_Guo2020',
 'ovary_ovary_HCLFetalFemaleGonad1',
 'ovary_ovary_HCLFetalFemaleGonad2',
 'prostate_prostate_HCLAdultProstate1',
 'testis_testis_HCLFetalMaleGonad1',
 'testis_testis_HCLFetalMaleGonad2',
 'uterinetube_uterinetube_HCLAdultFallopiantube1',
 'uterus_cervix_HCLAdultCervix1']

In [6]:
for organ in hcad_organs:
    df_update = pd.read_csv("/stor/public/hcad/db_v1/" + organ + "/" + organ + "df_update.csv")
    df_update = df_update.replace(np.NaN,"NA")
    rows_to_get = [[("cid",i)]for i in df_update['cid']]
    updates = [[("hcad_name",df_update['hcad_name'][j]),("cl_name","NA"),("tissue_type",df_update['tissue_type'][j]),("cell_type",df_update['cell_type'][j]),("marker_gene",df_update['marker_gene'][j])] for j in range(df_update.shape[0])]
    HCAd_Client.update_batch(rows_to_get,updates,thread_num = 30)
    df = HCAd_Client.get_columnsbycell_para(rows_to_get = rows_to_get, cols_to_get=["cell_type",'cl_name','hcad_name',"marker_gene","tissue_type"], col_filter=None, do_transfer = True, thread_num = multiprocessing.cpu_count()-1)
    